14

In [1]:
import numpy as np
from random import uniform

class Particle:
    def __init__(self, x, y, velocidad_angular):
        self.x = x
        self.y = y
        self.velocidad_angular = velocidad_angular

class ParticleSimulator:

    def __init__(self, particles):
        self.particles = particles

    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.velocidad_angular * v_x
                d_y = timestep * p.velocidad_angular * v_y
                p.x += d_x
                p.y += d_y

    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        velocidad_angular_i = np.array([p.velocidad_angular for p in self.particles])
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * velocidad_angular_i[:, np.newaxis] * v_i
            r_i += d_i
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

 
def benchmark(npart=1000, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)

if __name__ == "__main__":
    benchmark(npart=10, method='python')
    benchmark(npart=10, method='numpy')

In [2]:
from timeit import timeit

# Profiling
print(timeit("benchmark(1000, 'python')", globals=globals(), number=1))
print(timeit("benchmark(1000, 'numpy')", globals=globals(), number=1))

3.5270255730001736
0.30780913399985366


15

In [4]:
import numpy as np
import numexpr as ne
import time


def approximate_pi_numpy(n):
    np.random.seed(0)  # Seed for reproducibility
    x = np.random.rand(n)  # Generate n random points in the x
    y = np.random.rand(n)  # Generate n random points in the y
    inside_circle = x**2 + y**2 <= 1  # Check if points are inside the unit circle
    pi_approx = 4 * np.sum(inside_circle) / n  # Approximate Pi
    return pi_approx

def approximate_pi_numexpr(n):
    np.random.seed(0)  # Seed for reproducibility
    x = np.random.rand(n)  # Generate n random points in the x
    y = np.random.rand(n)  # Generate n random points in the y
    inside_circle = ne.evaluate("x**2 + y**2 <= 1")  # Check if points are inside the unit circle using numexpr
    pi_approx = 4 * np.sum(inside_circle) / n  # Approximate Pi
    return pi_approx

n = 10**7  # Number of points to generate

# Timing numpy
pi_numpy = approximate_pi_numpy(n)

# Timing numexpr
pi_numexpr = approximate_pi_numexpr(n)


In [8]:
#profiling
from timeit import timeit

%timeit -n10 -r10 approximate_pi_numpy(10**7)
%timeit -n10 -r10 approximate_pi_numexpr(10**7)

183 ms ± 17.1 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
113 ms ± 6.75 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
